# Práctica de Aprendizaje Automático – Modelo Final

**Asignatura:** Aprendizaje Automático  
**Curso:** 2024–2025  
**Trabajo práctico grupal**  
**Alumnas:** [Elena Recio Álvarez - 100495725], [Alejandra Castuera García - 100499172]  
**Entrega:** Modelo final entrenado y predicciones para el conjunto de competición  


In [1]:
import pandas as pd
from sklearn.model_selection import train_test_split

# Cargar dataset de entrenamiento
df = pd.read_csv("attrition_availabledata_07.csv.gz")

# Separar variables y target
X = df.drop(columns=["Attrition"])
y = df["Attrition"]

# Cargar dataset de competición (sin columna objetivo)
df_competition = pd.read_csv("attrition_competition_07.csv.gz")


## Entrenamiento del modelo final

Seleccionamos como mejor modelo la SVM con kernel RBF, tras ajuste de hiperparámetros (`C` y `gamma`) en el paso anterior.  
Entrenamos este modelo usando **todos los datos disponibles de entrenamiento**.


In [2]:
from sklearn.svm import SVC
from sklearn.pipeline import Pipeline
from sklearn.compose import ColumnTransformer
from sklearn.impute import SimpleImputer
from sklearn.preprocessing import StandardScaler, OneHotEncoder

# Columnas numéricas y categóricas
columnas_num = X.select_dtypes(include=["int64", "float64"]).columns.tolist()
columnas_cat = X.select_dtypes(include="object").columns.tolist()

# Preprocesado final
preprocessor = ColumnTransformer([
    ("num", Pipeline([
        ("imp", SimpleImputer(strategy="mean")),
        ("scaler", StandardScaler())
    ]), columnas_num),
    ("cat", Pipeline([
        ("imp", SimpleImputer(strategy="most_frequent")),
        ("onehot", OneHotEncoder(handle_unknown="ignore"))
    ]), columnas_cat)
])

# Modelo definitivo con hiperparámetros seleccionados
modelo_final = Pipeline([
    ("preprocessor", preprocessor),
    ("modelo", SVC(kernel="rbf", C=10, gamma=0.01, probability=True))  # ajusta si tus mejores eran otros
])

# Entrenar con TODO el set disponible
modelo_final.fit(X, y)


Pipeline(steps=[('preprocessor',
                 ColumnTransformer(transformers=[('num',
                                                  Pipeline(steps=[('imp',
                                                                   SimpleImputer()),
                                                                  ('scaler',
                                                                   StandardScaler())]),
                                                  ['hrs', 'absences',
                                                   'JobInvolvement',
                                                   'PerformanceRating',
                                                   'EnvironmentSatisfaction',
                                                   'JobSatisfaction',
                                                   'WorkLifeBalance', 'Age',
                                                   'DistanceFromHome',
                                                   'Education', 'EmployeeCount',
                                                   'EmployeeID', 'JobLevel',
                                                   'MonthlyIncome'...
                                                   'TrainingTimesLastYear',
                                                   'YearsAtCompany',
                                                   'YearsSinceLastPromotion',
                                                   'YearsWithCurrManager']),
                                                 ('cat',
                                                  Pipeline(steps=[('imp',
                                                                   SimpleImputer(strategy='most_frequent')),
                                                                  ('onehot',
                                                                   OneHotEncoder(handle_unknown='ignore'))]),
                                                  ['BusinessTravel',
                                                   'Department',
                                                   'EducationField', 'Gender',
                                                   'JobRole', 'MaritalStatus',
                                                   'Over18'])])),
                ('modelo', SVC(C=10, gamma=0.01, probability=True))])

In [3]:
import joblib

# Guardar el modelo entrenado
joblib.dump(modelo_final, "modelo_final.pkl")
print("Modelo final guardado como modelo_final.pkl")


Modelo final guardado como modelo_final.pkl


## Generar predicciones para el conjunto de competición

Cargamos el conjunto `attrition_competition_07.csv.gz`, aplicamos el modelo y guardamos las predicciones en un archivo `.csv` con formato de entrega.


In [4]:
# Predecir (etiquetas)
predicciones = modelo_final.predict(df_competition)

# Guardar en CSV
df_envio = pd.DataFrame({
    "id": df_competition.index,
    "Attrition": predicciones
})
df_envio.to_csv("predicciones.csv", index=False)

print("Predicciones guardadas en predicciones.csv")


Predicciones guardadas en predicciones.csv


In [5]:
df_envio["Attrition"].value_counts()


,count
Attrition,
No,1343
Yes,127
